So today what we are trying to do is to make an MCMC sampler that work for a generic prior distribution for a generic proposal distribution in a non define number of dimensions!
Should be difficult in some way, let's see what are we able to do.

Let's start by encoding the distribution that we want to analize:

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [93]:
'''
To be evaluated the change to pandas DataFrame to evaluate the mean and covariance, working with it should be simpler and faster
'''
class Likelihood:
    # We want to create a class that compute the likelihood as it is better to
    # maintain constants inside of it and because we can manage better each single function
    
    # Remember that in the init each single function is calculated only in the calling of the function
    # this means that the value are not recycled when the variable is called back 

    def __init__(self, dataset=None, mean=None, cov_mat=None):
        # Initialization with a dataset
        if dataset is not None:
            self.dataset = dataset
            self.mean = np.mean(dataset, axis=0)
            self.cov_mat = np.cov(dataset)

        # Initialization with cov_mat and mean
        elif cov_mat is not None and mean is not None:
            self.mean = mean
            self.cov_mat = cov_mat
        else:
            raise ValueError("Insufficient arguments provided for initialization.")

        self.inv_cov = np.linalg.inv(self.cov_mat)
        self.det_cov = np.linalg.det(self.cov_mat)

    def gauss_likelihood(self, x):
        #this is the mathod that make the likelihood, so this is essentially the one to recall inside of funtions
        assert x.shape == self.mean.shape, "Input dimension doesn't match the mean dimension. [type(mean) = np.array]"
        
        diff = x - self.mean
        exponent_term = -0.5 * np.dot(np.dot(diff, self.inv_cov), diff.T)
        likelihood = (1 / ((2 * np.pi) ** (len(x) / 2) * np.sqrt(np.absolute(self.det_cov)))) * np.exp(exponent_term)
        
        return likelihood

In [94]:
def proposal_distr(mean, cov_mat):
    # Select the proposed state (new guess) from a multivariate normal distribution
    #with mean defined and selected covariance, the starting cov_mat should be "circular".

    return np.random.multivariate_normal(mean, cov_mat)

In [95]:
def mcmc(curr_state, cov_mat, likelihood, proposal_distr):
    # This is simply the code to iterate. 
    # We write a code that work for a single point of the cycle and return that point if it is accepted.
    
    # The likelihood of the point we are already in
    curr_likeli = likelihood(curr_state)

    # The proposed new state and the likelihood we are sitting on
    proposal_state = proposal_distr(curr_state, cov_mat)
    proposal_likeli = likelihood(proposal_state)

    # The acceptance parameter for the new state
    acceptance = proposal_likeli/curr_likeli

    
    trial = np.random.uniform(0,1)
    if np.all(np.greater_equal(acceptance, trial)):
        return proposal_state, proposal_likeli
    
    return curr_state, curr_likeli

In [100]:
def metropolis_hasting(initial_state, initial_cov, likelihood, proposal_distr, Nsteps):
    # Metropolis hastings variables initialization
    sample = []
    iterative = 0

    # State initialization
    curr_state = initial_state
    previous_likeli = likelihood(initial_state)
    curr_likeli = previous_likeli

    # Covariance initialization
    cov_mat = initial_cov

    for i in range(Nsteps):
        curr_state, curr_likeli= mcmc(curr_state = curr_state, cov_mat = cov_mat, likelihood = likelihood(), proposal_distr = proposal_distr)
        iterative +=1
        if curr_likeli is not previous_likeli:
            sample.append([curr_state, curr_likeli, iterative])
            iterative=0
    
    return sample

You should update the covariance matrix making a Gelman-Rubin convergence and by setting a value of stop updating for like 3>R>0.3 or as you find usefull.

In [ ]:
def GelmanRubin :
    R #cumputation
    #evaluation of the point of convegence
    #update covariance
  
    

In [103]:
# Let us write the actual code of sampling (potato mode)

target_covariance = np.array([[1, 2, 3], [4, 1, 5], [8, 2, 1]]).T
target_mean = np.array([4, 1, 18])

likeli = Likelihood(mean= target_mean, cov_mat= target_covariance)
likeli.gauss_likelihood(np.array([1,1,1]).T)

proposal_mean = np.array([1, 1, 1])
proposal_covariance = np.array([[1, 2, 3], [2, 1, 4], [3, 4, 1]]).T

proposal = proposal_distr(mean= proposal_mean, cov_mat= proposal_covariance)
Nsteps = 100

simulated_data = metropolis_hasting(initial_state= proposal_mean, initial_cov= proposal_covariance, likelihood= likeli.gauss_likelihood(proposal_mean), proposal_distr= proposal, Nsteps=Nsteps)
simulated_data

/tmp/ipykernel_15495/154114418.py:5: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  return np.random.multivariate_normal(mean, cov_mat)


TypeError: 'numpy.float64' object is not callable

In [ ]:
# Setup of the target distribution and number of steps
covariance = np.array([[1, -3], [-6, 1]]).T

a = Likelihood(mean = np.array([3., 5.]), cov_mat= covariance)
print(f'mean = {a.mean}')
type(a.mean)

y = np.array([1. ,1. ])
print(y)

b = a.gauss_likelihood(y)
print(b)
covariance = np.array([[3, 3], [-9, 1]])
muP = np.array([4, 4])
Nsteps = 10000

In [21]:
df = pd.DataFrame(sample)
df

,0,1
0,4.000000,4.000000
1,4.000000,4.000000
2,4.000000,4.000000
3,4.000000,4.000000
4,4.000000,4.000000
...,...,...
9995,2.276190,4.230331
9996,2.276190,4.230331
9997,3.797948,5.343338
9998,3.309655,5.542114
